In [1]:
import numpy as np
import torch

from torchvision import models
from torchvision import transforms
import torch.nn as nn
import torchvision.models as models

from pathlib import Path
from collections import Counter

from tqdm import tqdm
from torch.utils.data import DataLoader

from PIL import Image
import numpy as np

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [8]:
from pathlib import Path

try:
    Path( Path().resolve().parent / "data" /  "train" / "high jump" / "159.lnk").unlink()
except FileNotFoundError:
    pass

In [9]:
import os
from torch.utils.data import Dataset
from PIL import Image

class SportPictureDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.classes = sorted(os.listdir(data_dir))
        self.class_to_idx = {cls: i for i, cls in enumerate(self.classes)}

        for label_name in self.classes:
            label_dir = os.path.join(data_dir, label_name)
            for img_name in os.listdir(label_dir):
                self.image_paths.append(os.path.join(label_dir, img_name))
                self.labels.append(self.class_to_idx[label_name])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

In [10]:
from torchvision import transforms


# Simple transforms - just resize and normalize
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomRotation(10),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ) # ImageNet normalization
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ) # ImageNet normalization
])


test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ) # ImageNet normalization
])

In [11]:
import torch.nn as nn
import torchvision.models as models

class SportImageClassifierEfficientNet(nn.Module):
    def __init__(self, num_classes=100, dropout_rate=0.3):
        super(SportImageClassifierEfficientNet, self).__init__()

        # Load pre-trained Efficient Net
        self.base_model = models.efficientnet_b0(weights='IMAGENET1K_V1')

        # Freeze base model parameters
        for param in self.base_model.parameters():
            param.requires_grad = False

        # Remove original classifier
        self.base_model.classifier = nn.Identity()

        # Add custom layers
        self.classifier = nn.Sequential(
            nn.Dropout(p=dropout_rate),
            nn.Linear(1280, 512),
            nn.ReLU(),
            nn.Dropout(p=dropout_rate),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.base_model(x)
        x = self.classifier(x)
        return x

In [12]:
from itertools import product
param_grid = {
    'weight_decay': [0.001, 0.01, 0.1], 'lr': [0.001, 0.01], 'dropout_rate': [0.2, 0.3], 'batch_size': [128, 64],      
    
}


keys = param_grid.keys()
values = param_grid.values()
hyperparam_combinations = [dict(zip(keys, v)) for v in product(*values)]

In [13]:
len(hyperparam_combinations)

24

In [14]:
train_dataset = SportPictureDataset(
    data_dir= Path( Path().resolve().parent / "data" /  "train"),
    transform=train_transforms
)

val_dataset = SportPictureDataset(
    data_dir= Path( Path().resolve().parent / "data" /  "valid"),
    transform=val_transforms
)


test_dataset = SportPictureDataset(
    data_dir= Path( Path().resolve().parent / "data" /  "test"),
    transform=val_transforms
)

In [15]:
import torch
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = nn.CrossEntropyLoss()

In [16]:
def hyperparam_gridsearch(hyperparam_dict, num_epochs = 10):
  print(hyperparam_dict)
  train_loader = DataLoader(train_dataset, batch_size=hyperparam_dict['batch_size'], shuffle=True)
  val_loader = DataLoader(val_dataset, batch_size=hyperparam_dict['batch_size'], shuffle=False)

  model = SportImageClassifierEfficientNet(num_classes=100, dropout_rate=hyperparam_dict['dropout_rate'])
  model.to(device)
  optimizer = optim.AdamW( filter(lambda p: p.requires_grad, model.parameters()), lr=hyperparam_dict['lr'], weight_decay=hyperparam_dict['weight_decay'])
  scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2)

  num_epochs = num_epochs
  best_val_acc = 0.0
  patience = 5 
  patience_counter = 0

  train_losses = []
  train_accs = []
  val_losses = []
  val_accs = []

  for epoch in range(num_epochs):
    # Training phase
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    train_pbar = tqdm(train_loader, desc='Training', leave=False)

    for inputs, labels in train_pbar:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        # Update progress bar
        train_pbar.set_postfix({'loss': loss.item(), 'acc': correct/total})

    train_loss = running_loss / len(train_loader)
    train_acc = correct / total

    # Validation phase
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    # Progress bar for validation
    val_pbar = tqdm(val_loader, desc='Validation', leave=False)

    with torch.no_grad():
        for inputs, labels in val_pbar:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()
            
            # Update progress bar
            val_pbar.set_postfix({'loss': loss.item(), 'acc': val_correct/val_total})

    val_loss /= len(val_loader)
    val_acc = val_correct / val_total

    print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}')
    print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')

    scheduler.step(val_acc)

    train_losses.append(train_loss)
    train_accs.append(train_acc)
    val_losses.append(val_loss)
    val_accs.append(val_acc)

    print(f'Epoch {epoch+1}/{num_epochs} - Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')


    if val_acc > best_val_acc:
        best_val_acc = val_acc
        patience_counter = 0

        
    else:
      patience_counter += 1

    if patience_counter >= patience:
        print(f'\nEarly stopping triggered after {epoch+1} epochs')
        print(f'Best validation accuracy: {best_val_acc:.4f}')
        break





In [18]:
for hyperparam_dict in hyperparam_combinations:
  hyperparam_gridsearch(hyperparam_dict)

{'weight_decay': 0.01, 'lr': 0.01, 'dropout_rate': 0.2, 'batch_size': 128}
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth


100%|██████████| 20.5M/20.5M [00:00<00:00, 145MB/s] 


Train Loss: 1.9644, Train Acc: 0.5032
Val Loss: 0.8334, Val Acc: 0.7380
Epoch 1/10 - Train Loss: 1.9644, Train Acc: 0.5032, Val Loss: 0.8334, Val Acc: 0.7380


Train Loss: 1.3281, Train Acc: 0.6570
Val Loss: 0.8276, Val Acc: 0.7840
Epoch 2/10 - Train Loss: 1.3281, Train Acc: 0.6570, Val Loss: 0.8276, Val Acc: 0.7840


Train Loss: 1.2610, Train Acc: 0.6784
Val Loss: 0.7975, Val Acc: 0.7900
Epoch 3/10 - Train Loss: 1.2610, Train Acc: 0.6784, Val Loss: 0.7975, Val Acc: 0.7900


...

Train Loss: 1.7589, Train Acc: 0.5789
Val Loss: 1.0058, Val Acc: 0.7300
Epoch 6/10 - Train Loss: 1.7589, Train Acc: 0.5789, Val Loss: 1.0058, Val Acc: 0.7300


Train Loss: 1.3441, Train Acc: 0.6539
Val Loss: 0.6908, Val Acc: 0.8100
Epoch 7/10 - Train Loss: 1.3441, Train Acc: 0.6539, Val Loss: 0.6908, Val Acc: 0.8100


Train Loss: 1.2076, Train Acc: 0.6753
Val Loss: 0.6716, Val Acc: 0.8060
Epoch 8/10 - Train Loss: 1.2076, Train Acc: 0.6753, Val Loss: 0.6716, Val Acc: 0.8060


Train Loss: 1.1321, Train Acc: 0.6957
Val Loss: 0.6379, Val Acc: 0.8180
Epoch 9/10 - Train Loss: 1.1321, Train Acc: 0.6957, Val Loss: 0.6379, Val Acc: 0.8180


Train Loss: 1.0814, Train Acc: 0.7028
Val Loss: 0.5739, Val Acc: 0.8380
Epoch 10/10 - Train Loss: 1.0814, Train Acc: 0.7028, Val Loss: 0.5739, Val Acc: 0.8380


In [ ]:
def final_model_backpropogation(hyperparam_dict, num_epochs = 10):
    print(hyperparam_dict)
    train_loader = DataLoader(train_dataset, batch_size=hyperparam_dict['batch_size'], shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=hyperparam_dict['batch_size'], shuffle=False)
    
    model = SportImageClassifierEfficientNet(num_classes=100, dropout_rate=hyperparam_dict['dropout_rate'])
    model.to(device)
    optimizer = optim.AdamW( filter(lambda p: p.requires_grad, model.parameters()), lr=hyperparam_dict['lr'], weight_decay=hyperparam_dict['weight_decay'])
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2)
    
    num_epochs = num_epochs
    best_val_acc = 0.0
    patience = 5 
    patience_counter = 0
    
    train_losses = []
    train_accs = []
    val_losses = []
    val_accs = []
    
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        train_pbar = tqdm(train_loader, desc='Training', leave=False)
        
        for inputs, labels in train_pbar:
            inputs, labels = inputs.to(device), labels.to(device)
        
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            # Update progress bar
            train_pbar.set_postfix({'loss': loss.item(), 'acc': correct/total})
        
        train_loss = running_loss / len(train_loader)
        train_acc = correct / total
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        
        # Progress bar for validation
        val_pbar = tqdm(val_loader, desc='Validation', leave=False)
        
        with torch.no_grad():
            for inputs, labels in val_pbar:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
        
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
                
                # Update progress bar
                val_pbar.set_postfix({'loss': loss.item(), 'acc': val_correct/val_total})
        
        val_loss /= len(val_loader)
        val_acc = val_correct / val_total
        
        print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}')
        print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')
        
        scheduler.step(val_acc)
        
        train_losses.append(train_loss)
        train_accs.append(train_acc)
        val_losses.append(val_loss)
        val_accs.append(val_acc)
        
        print(f'Epoch {epoch+1}/{num_epochs} - Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')
        
        
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            patience_counter = 0
        
            torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'hyperparameters': hyperparam_dict,
                    'val_acc': val_acc,
                    'val_loss': val_loss,
                }, 'best_model.pth')
        
            #model_name = f"model_lr{hyperparam_dict['lr']}_wd{hyperparam_dict['weight_decay']}_dr{hyperparam_dict['dropout_rate']}_bs{hyperparam_dict['batch_size']}_val_acc{val_acc:.3f}.onnx"

        
            # torch.save({
            #     'epoch': epoch,
            #     'model_state_dict': model.state_dict(),
            #     'optimizer_state_dict': optimizer.state_dict(),
            #     'val_acc': val_acc,
            #     'val_loss': val_loss,
            # }, 'best_model.pth')
            # dummy_input = torch.randn(1, 3, 224, 224).to(device)
        
            # torch.onnx.export(
            #     model,
            #     dummy_input,
            #     model_name,
            #     verbose=True,
            #     input_names=['input'],
            #     output_names=['output'],
            #     dynamic_axes={
            #         'input': {0: 'batch_size'},
            #         'output': {0: 'batch_size'}
            #     }
            # )
        else:
          patience_counter += 1
        
        if patience_counter >= patience:
            print(f'\nEarly stopping triggered after {epoch+1} epochs')
            print(f'Best validation accuracy: {best_val_acc:.4f}')
            break
        torch.cuda.empty_cache()
        
        
    # Load best model
    checkpoint = torch.load('best_model.pth')
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()
    
    # Export to ONNX
    model_name = f"best_sport-image-classifier-model.onnx"
    
    dummy_input = torch.randn(1, 3, 224, 224).to(device)

    
    try:
        import warnings
        with warnings.catch_warnings():
            warnings.filterwarnings('ignore', category=DeprecationWarning)
            torch.onnx.export(
                model,
                dummy_input,
                model_name,
                verbose=False,
                input_names=['input'],
                output_names=['output'],
                dynamic_axes={
                    'input': {0: 'batch_size'},
                    'output': {0: 'batch_size'}
                }
            )
        print(f'✓ ONNX model saved: {model_name}')
    except Exception as e:
        print(f'⚠ Failed to save ONNX: {e}')
    
    
    return {
    'best_val_acc': best_val_acc,
    'train_losses': train_losses,
    'train_accs': train_accs,
    'val_losses': val_losses,
    'val_accs': val_accs
    }





In [18]:
# choose the most optimal setup and do 30 epochs
final_hyperparam_combos = [{'weight_decay': 0.01, 'lr': 0.001, 'dropout_rate': 0.2, 'batch_size': 128}]


for hyperparam_dict in final_hyperparam_combos:
  final_model_backpropogation(hyperparam_dict, num_epochs=30)


{'weight_decay': 0.01, 'lr': 0.001, 'dropout_rate': 0.2, 'batch_size': 128}
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth


100%|██████████| 20.5M/20.5M [00:00<00:00, 115MB/s] 


Train Loss: 2.6968, Train Acc: 0.4307
Val Loss: 1.0873, Val Acc: 0.7320
Epoch 1/30 - Train Loss: 2.6968, Train Acc: 0.4307, Val Loss: 1.0873, Val Acc: 0.7320


Train Loss: 1.0974, Train Acc: 0.7227
Val Loss: 0.6370, Val Acc: 0.8460
Epoch 2/30 - Train Loss: 1.0974, Train Acc: 0.7227, Val Loss: 0.6370, Val Acc: 0.8460


Train Loss: 0.8439, Train Acc: 0.7765
Val Loss: 0.5191, Val Acc: 0.8700
Epoch 3/30 - Train Loss: 0.8439, Train Acc: 0.7765, Val Loss: 0.5191, Val Acc: 0.8700


...

Train Loss: 0.3340, Train Acc: 0.9028
Val Loss: 0.3398, Val Acc: 0.9040
Epoch 15/30 - Train Loss: 0.3340, Train Acc: 0.9028, Val Loss: 0.3398, Val Acc: 0.9040


Train Loss: 0.3186, Train Acc: 0.9097
Val Loss: 0.3336, Val Acc: 0.9000
Epoch 16/30 - Train Loss: 0.3186, Train Acc: 0.9097, Val Loss: 0.3336, Val Acc: 0.9000


Train Loss: 0.3084, Train Acc: 0.9114
Val Loss: 0.3462, Val Acc: 0.8960
Epoch 17/30 - Train Loss: 0.3084, Train Acc: 0.9114, Val Loss: 0.3462, Val Acc: 0.8960


Train Loss: 0.2898, Train Acc: 0.9168
Val Loss: 0.3265, Val Acc: 0.9000
Epoch 18/30 - Train Loss: 0.2898, Train Acc: 0.9168, Val Loss: 0.3265, Val Acc: 0.9000


Train Loss: 0.2882, Train Acc: 0.9194
Val Loss: 0.3190, Val Acc: 0.8980
Epoch 19/30 - Train Loss: 0.2882, Train Acc: 0.9194, Val Loss: 0.3190, Val Acc: 0.8980

Early stopping triggered after 19 epochs
Best validation accuracy: 0.9080
✓ ONNX model saved: best_model_lr0.001_wd0.01_dr0.2_bs128_val_acc0.908.onnx


In [ ]:
import onnxruntime as ort
import numpy as np

ort_session = ort.InferenceSession(r'best_sport-image-classifier-model.onnx')
input_name = ort_session.get_inputs()[0].name

In [24]:
test_loader = DataLoader(
        test_dataset,
        batch_size=hyperparam_dict['batch_size'],
        shuffle=False,
        num_workers=2,
        pin_memory=True
    )

In [25]:
# Test
test_correct = 0
test_total = 0
all_predictions = []
all_labels = []

print("Evaluating ONNX model on test set...")
for inputs, labels in tqdm(test_loader, desc='Testing'):
    # Convert to numpy
    inputs_np = inputs.numpy()
    
    # Run inference
    outputs = ort_session.run(None, {input_name: inputs_np})
    predictions = np.argmax(outputs[0], axis=1)
    
    test_total += labels.size(0)
    test_correct += (predictions == labels.numpy()).sum()
    
    all_predictions.extend(predictions.tolist())
    all_labels.extend(labels.numpy().tolist())

test_acc = test_correct / test_total

print(f"\n{'='*60}")
print(f"ONNX MODEL TEST SET RESULTS")
print(f"{'='*60}")
print(f"Test Accuracy: {test_acc:.4f} ({test_acc*100:.2f}%)")
print(f"Correct: {test_correct}/{test_total}")
print(f"{'='*60}")

Evaluating ONNX model on test set...


Testing: 100%|██████████| 4/4 [00:09<00:00,  2.39s/it]


ONNX MODEL TEST SET RESULTS
Test Accuracy: 0.9260 (92.60%)
Correct: 463/500
